In [1]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
# Metrics
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

from datetime import datetime
import math

# Loading Data

In [2]:
movie_lines_path = 'movie_lines.txt'
movie_conversations_path = 'movie_conversations.txt'

In [3]:
with open(movie_lines_path, encoding='iso-8859-1', errors='ignore') as my_file:
    all_lines = {}
    for line in my_file:
        split = line.split(' +++$+++ ')
        linemp = {}
        fields = ["lineID", "characterID", "movieID", "character", "text"]
        count = 0
        for field in (fields):
                linemp[field] = split[count]
                count +=1
        all_lines[linemp['lineID']] = linemp        
        


In [4]:
with open(movie_conversations_path, encoding='iso-8859-1', errors='ignore') as my_file:
    conv = []
    for line in my_file:
        split = line.split(' +++$+++ ')
        obj = {}
        fields = ["character1ID", "character2ID", "movieID", "utteranceIDs"]
        count = 0 
        for field in fields:
            obj[field] = split[count]
            count +=1
        ID = re.compile('L[0-9]+').findall(obj['utteranceIDs'])
        lines = []
        
        for id_ in ID:
            lines.append(all_lines[id_])
        obj['line'] = lines
        conv.append(obj)

In [5]:
all_lines["L985"]

{'lineID': 'L985',
 'characterID': 'u0',
 'movieID': 'm0',
 'character': 'BIANCA',
 'text': 'I hope so.\n'}

In [6]:
conv[10]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L367', 'L368']\n",
 'line': [{'lineID': 'L367',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': 'How do you get your hair to look like that?\n'},
  {'lineID': 'L368',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': "Eber's Deep Conditioner every two days. And I never, ever use a blowdryer without the diffuser attachment.\n"}]}

# Matching Data

In [7]:
pairs = []
for convrtsation in conv:
        for i in range(len(convrtsation['line'])):
            try:
                question = convrtsation['line'][i]['text'].strip()
                answer = convrtsation['line'][i+1]['text'].strip()
            except:
                pass
            if(question and answer):
                pairs.append([question, answer])

In [8]:
len(pairs)

304309

In [9]:
for i in range (10):
    print(pairs[i])

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.', "Well, I thought we'd start with pronunciation, if that's okay with you."]
["Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part.  Please.']
['Not the hacking and gagging and spitting part.  Please.', "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]
["Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?", "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]
["You're asking me out.  That's so cute. What's your name again?", 'Forget it.']
['Forget it.', 'Forget it.']
["No, no, it's my fault -- we didn't have a proper introduction ---", 'Cameron.']
['Cameron.', "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."]
["The thing is, Cameron --

In [10]:
class Vocab:
    def __init__(self):
        self.enum = {"PAD_token" : 0, "SOS_token" : 1, "EOS_token":2, "UNK":3}
        self.count = {}
        self.index = {}
        self.wordcount = 4
        self.min_freq = 3
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            if word not in self.enum:
                if(word in self.count.keys()):
                    self.count[word] += 1
                    if(self.count[word] >= self.min_freq):
                        self.enum[word] = self.wordcount
                        self.index[self.wordcount] = word
                        self.wordcount += 1
                else:
                    self.count[word] = 1
            else:
                #print("Word already Added")
                self.count[word] += 1
    def __len__(self):
        return self.wordcount    
                
    ### This will be the class that handles the bag of words.
    

In [11]:
PAD_token = 0
SOS_token = 1
EOS_token = 2
UNK = 3

In [12]:
voc = Vocab()

In [13]:
import numpy as np

In [14]:
shape = np.array(pairs)
shape.shape

(304309, 2)

In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [16]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
import string
def clean_String(stri):
    new_string = ''
    for i in stri:
        if i not in string.punctuation:
            new_string += i
    stri = new_string
    
    lower_string = stri.lower()
    no_number_string = re.sub(r'\d+','',lower_string)
    no_punc_string = re.sub(r'[^\w\s]','', no_number_string) 
    no_wspace_string = no_punc_string.strip()
    
    words = no_wspace_string.split()
    #filtered_words = [word for word in words if word not in stop_words]
    # I am unsure if removing stop words is correct on a chat bot for readability reasons
    #

        
    return ' '.join(words)

In [18]:
for i in pairs:
    for j in i:
        cleaned = clean_String(j)
        voc.addSentence(cleaned)

In [19]:
len(voc)

42350

In [20]:
max_len = 50

In [21]:
pairs[11]

['Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame."]

# Enumerating the Data

In [22]:
def enc_ques(words, voc):
    words = clean_String(words)
    words = (words)
    encoded = []
    count = 0
    for word in words.split(' '):
        encoded.append(voc.enum.get(word, voc.enum['UNK']))
        count = count + 1
        if(count == 30):
            break
    while(count < 30):
        count = count + 1
        encoded.append(voc.enum['PAD_token'])
    
                                
    if(not len(encoded) == 30):
        print(len(encoded))
    return encoded

In [23]:
def enc_rep(words, voc):
    words = clean_String(words)
    
    encoded = []
    encoded.append(voc.enum['SOS_token'])
    count = 1
    for word in words.split(' '):
        
        if(count == 29):
            break
        encoded.append(voc.enum.get(word, voc.enum['UNK']))
        count = count + 1
    while(count < 29):
        count = count + 1
        encoded.append(voc.enum['PAD_token'])
    
    encoded.append(voc.enum['EOS_token'])
    if(not len(encoded) == 30):
        print(len(encoded))
    return encoded

In [24]:
pairs_encoded = []
for pair in pairs:
    qus = enc_ques(pair[0], voc)
    ans = enc_rep(pair[1], voc)
    #print(qus, ans)
    pairs_encoded.append([qus, ans])
    

In [25]:
class Dataset(Dataset):

    def __init__(self, pairs):
        self.pairs = pairs
    def __getitem__(self, i):
        
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
            
        return question, reply

    def __len__(self):
        return len(pairs)

In [26]:
len(pairs)

304309

In [27]:
dataset = Dataset(pairs_encoded)

In [28]:
304309-280000

24309

In [29]:
dataset[11]

(tensor([13807,  5432,    27,   356,    45,    55,    89,  1142,   171,    27,
          1405,   263,   264,     9,    21,   133,   155,    30,    27,    61,
           986,    24,    21,   173,    62,     0,     0,     0,     0,     0]),
 tensor([ 1, 19, 25, 35,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2]))

In [30]:
np.array(pairs).shape

(304309, 2)

In [31]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [280000, 24309])

In [32]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 100)

# Models

In [33]:
#Based off of https://medium.com/analytics-vidhya/bert-implementation-multi-head-attention-4a10142636fe

class Embedding(nn.Module):
        def __init__(self, voc_size , size, max_len = 30):
            super(Embedding, self).__init__()
            self.divs = 10000
            self.size = size
            self.dropout = nn.Dropout(0.1)
            self.embed = nn.Embedding(voc_size, size)
            self.out = self.pos_enc(max_len, self.size)
        
        def calc(self, out, size, pos, loc):
            out[pos, loc] = math.sin(pos / (self.divs ** ((2 * loc)/size)))
            out[pos, loc + 1] = math.cos(pos / (self.divs ** ((2 * (loc + 1))/size)))
            #print(out)
            return out
            
        def pos_enc(self, max_len, size):
            out = torch.zeros(max_len, size).to(device)
            
            for pos in range(max_len):  
                for loc in range(math.ceil(size/2)):
                    loc = loc * 2
                    out = self.calc(out, size, pos, loc)
            out = out.unsqueeze(0)   
            #print(out)
            return out
        def forward(self, enc_out):
            emb = self.embed(enc_out) * math.sqrt(self.size)
            emb += self.out[:, :emb.size(1)]  
            emb = self.dropout(emb)
            return emb
        

In [34]:
class Attn(nn.Module):
    #Based off of https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html

    def scaled_dot_product(self, q, k, v, mask=None):
        
        sear = q.view(q.shape[0], -1, self.num_heads, self.divisor).permute(0, 2, 1, 3)   
        q = k.view(q.shape[0], -1, self.num_heads, self.divisor).permute(0, 2, 1, 3)  
        v = v.view(v.shape[0], -1, self.num_heads, self.divisor).permute(0, 2, 1, 3)  

        score = torch.matmul(sear, q.permute(0,1,3,2)) / math.sqrt(sear.size(-1))
        score = score.masked_fill(mask == 0, -1e9)    
        weights = F.softmax(score, dim = -1)          
        weights = self.dropout(weights)
        product = torch.matmul(weights, v)
        
        return product
        


    def __init__(self, num_heads, size):

        super(Attn, self).__init__()
        self.num_heads = num_heads
        self.dropout = nn.Dropout(0.1)
        self.query, self.key, self.value, self.concat  = nn.Linear(size, size), nn.Linear(size, size), nn.Linear(size, size), nn.Linear(size, size)
        self.divisor = int(size / num_heads)

    def forward(self, search, key, value, mask):
        search = self.query(search)
        value = self.value(value)
        search_key = self.key(key)
        product = self.scaled_dot_product(search, search_key, value, mask) 
    
        product = product.permute(0,2,1,3).contiguous().view(product.shape[0], -1, self.num_heads * self.divisor)
        interacted = self.concat(product)
        return interacted 

In [35]:
#Based off of https://medium.com/analytics-vidhya/bert-implementation-multi-head-attention-4a10142636fe

class Model(nn.Module):
        
    class Ann(nn.Module):

        def __init__(self, size):
            super(Model.Ann, self).__init__()
            
            self.fc1 = nn.Linear(size, 2048)
            self.fc3 = nn.Linear(2048 , size)
            self.dropout = nn.Dropout(0.1)

        def forward(self, x):
            out = F.relu(self.fc1(x))
            #out = self.fc2(self.dropout(out))
            out = self.fc3(self.dropout(out))
            return out

    class Encoder(nn.Module):

        def __init__(self, size, heads):
            super(Model.Encoder, self).__init__()
            self.layernorm = nn.LayerNorm(size)
            self.attn = Attn(heads, size)
            self.an = Model.Ann(size)
            self.dropout = nn.Dropout(0.1)

        def forward(self, emb, mask):
            atten = self.dropout(self.attn(emb, emb, emb, mask))
            atten = self.layernorm(atten + emb)
            out = self.dropout(self.an(atten))
            encoded = self.layernorm(out + atten)
            return encoded
    
    
    
    class Decoder(nn.Module):
        def __init__(self, size, heads):
            super(Model.Decoder, self).__init__()
            self.an = Model.Ann(size)
            self.dropout = nn.Dropout(0.1)
            
            self.layernorm = nn.LayerNorm(size)
            
            self.attn = Attn(heads, size)
            self.attb = Attn(heads, size)
            
        def forward(self, emb, encoded, s_mask, t_mask):
            search = self.dropout(self.attn(emb, emb, emb, t_mask))
            search = self.layernorm(search + emb)
            atten = self.dropout(self.attb(search, encoded, encoded, s_mask))
            atten = self.layernorm(atten + search)
            out = self.dropout(self.an(atten))
            decoded = self.layernorm(out + atten)
            return decoded
    
    def __init__(self, size, heads):
        super(Model, self).__init__()
        
        self.size = size
        self.vocab_size = len(voc)
        self.embed = Embedding(self.vocab_size, size)
        list_dec = []
        list_enc = []
        for i in range(3):
            list_enc.append(self.Encoder(size, heads))
            list_dec.append(self.Decoder(size, heads))
        
        self.encoder = nn.ModuleList(list_enc)
        self.decoder = nn.ModuleList(list_dec)
        
        self.logit = nn.Linear(size, self.vocab_size)
    
    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
        return src_embeddings
    
    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings
        
    def forward(self, word, s_mask, t_word, t_mask):
        s_emb = self.encode(word, s_mask)
        decoded = self.decode(t_word, t_mask, s_emb, s_mask)
        
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out


In [36]:
class loss(nn.Module):

    def __init__(self, size):
        super(loss, self).__init__()
        self.size = size
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        
    def label(self,pred, target):
        labels = pred.data.clone()
        labels.fill_(.1 / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), .9)
        
        return labels
    
    def loss(self, pred, labels , mask):
        loss = self.criterion(pred, labels)   
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss
        
    def forward(self, pred, target, mask):

        pred = pred.view(-1, pred.size(-1)) 
        target = target.contiguous().view(-1)  
        mask = mask.float().view(-1)
        labels = self.label(pred, target)
        loss = self.loss(pred, labels , mask)
        return loss

In [37]:
#From /questions/65343377/adam-optimizer-with-warmup-on-pytorch
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.model_size = model_size
        self._rate = 0
    
    def state_dict(self):
        """Returns the state of the warmup scheduler as a :class:`dict`.
        It contains an entry for every variable in self.__dict__ which
        is not the optimizer.
        """
        return {key: value for key, value in self.__dict__.items() if key != 'optimizer'}
    
    def load_state_dict(self, state_dict):
        """Loads the warmup scheduler's state.
        Arguments:
            state_dict (dict): warmup scheduler state. Should be an object returned
                from a call to :meth:`state_dict`.
        """
        self.__dict__.update(state_dict) 
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5))) 

In [39]:
size = 512
heads = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 30
    

model = Model(size = size, heads = heads)
model = model.to(device)
optimizer = NoamOpt(size, 500,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))



criterion = loss(len(voc))

E:\Ana\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [40]:
epochs = 30


In [41]:
def processor(size):
    mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
    return mask.unsqueeze(0)

In [42]:
def process_inputs(question, reply):
        rep_inp = reply[:, :-1]
        reply_target = reply[:, 1:]
        
        reply_input_mask = rep_inp
        reply_input_mask = reply_input_mask.unsqueeze(1)
        reply_target_mask = reply_target!=0 
        #print(reply_input_mask)
        reply_input_mask = torch.bitwise_and(reply_input_mask, processor(rep_inp.size(-1)).to(device) ).type_as(reply_input_mask.data) 
        #print(reply_input_mask)
        reply_input_mask = reply_input_mask.unsqueeze(1) 
        question_mask = question
        question_mask = question_mask.to(device)
        question_mask = question_mask.unsqueeze(1).unsqueeze(1)
        
        return reply_target, question_mask, rep_inp, reply_input_mask, reply_target_mask

In [47]:
def train(train_loader, model, criterion, epoch):
    
    model.train()
    sum_loss = 0
    count = 0

    for i, (question, answer) in enumerate(train_loader):
        
        samples = question.shape[0]

        question = question.to(device)
        answer = answer.to(device)
            
        answer_T, question_mask, answer_I, answer_IM, answer_tM = process_inputs(question, answer)
        
        out = model(question, question_mask, answer_I, answer_IM)
        
        loss = criterion(out, answer_T, answer_tM)
        
        
        optimizer.optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item() * samples
        count += samples
        
        
        if i % 100 == 0:
            time = datetime.now()
            print("Epoch : {} Batch : {}/{} Time {} ]\tLoss: {:.3f}".format(epoch, i, len(train_loader), time, sum_loss/count,))
            #print(out)
            #print(reply_target)

In [48]:
for epoch in range(epochs):
    
    train(train_loader, model, criterion, epoch)
    
    state = {'epoch': epoch, 'transformer': model}
    torch.save(state, 'checkpoint_' + str(epoch) + '.pth.tar')

Epoch : 0 Batch : 0/2800 Time 2023-10-20 23:48:17.069740 ]	Loss: 9.412
Epoch : 0 Batch : 100/2800 Time 2023-10-20 23:51:20.501979 ]	Loss: 6.448


KeyboardInterrupt: 